# Búsqueda semántica con nubes de puntos
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \frac{1}{2|U|} \sum_{u \in U} \min_{x \in V} d(u, x) + \frac{1}{2|V|}\sum_{v \in V} \min_{x \in U} d(v, x) $$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{tp}$

$$ H^\textsf{pt}_d(U, V) = \max \left\{
        \max_{u \in U} \left\{\textsf{pt}(u, U) \min_{x \in V} d(u, x)\right\},
        \max_{v \in V} \left\{\textsf{pt}(v, V) \min_{x \in U} d(v, x)\right\}
     \right\} $$

# Notas adicionales
La versión de $H^\textsf{pt}_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, JSON, CodecZlib, Random
using Embeddings: load_embeddings, FastText_Text
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
include("knn.jl")

scores (generic function with 1 method)

In [3]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 10000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

# Cargando datos y creando word embeddings a la medida

In [4]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=false, group_url=true, group_num=true, del_diac=true, del_punc=true)

display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
embfile = "../data/news.w2v.vec"
isfile(embfile) && rm(embfile)
word2vec_embeddings(textconfig, D.corpus, embfile; dim=100, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)

@show unique(D.labels)

<h1>Cargando el corpus</h1>

Starting training using file /home/sadit/IR-2022/Unidades/w2v.tmp
Vocab size: 11080
Words in train file: 455119
unique(D.labels) = ["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"]


25-element Vector{String}:
 "AdriDelgadoRuiz"
 "El_Universal_Mx"
 "CNNEE"
 "NTN24"
 "UniNoticias"
 "TelemundoNews"
 "SinEmbargoMX"
 "Reforma"
 "abc_es"
 "azucenau"
 "AristeguiOnline"
 "el_pais"
 "EFEnoticias"
 "PublimetroMX"
 "PublimetroChile"
 "Radio_Formula"
 "RicardoAlemanMx"
 "epigmenioibarra"
 "Milenio"
 "LaRazon_mx"
 "abrahamendieta"
 "PublimetroCol"
 "teleSURtv"
 "bbcmundo"
 "julioastillero"

In [5]:
T = let
    voc = Vocabulary(textconfig, D.corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

(textconfig = TextConfig(true, false, true, true, true, false, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=11080, train-n=30244, maxoccs=43239}, voc = Vocabulary(["</s>", "los", "del", "por", "un", "rt", "se", "con", "las", "para"  …  "marbelle", "cruces", "nolleva", "acaboelpaseviernes", "@docede", "@gregoriacarod", "salvadorena", "ignorancia", "nombrado", "mayorkas"], Int32[0, 8882, 7593, 6956, 6568, 6721, 6328, 5999, 5963, 5070  …  6, 6, 5, 5, 5, 5, 5, 5, 5, 5], Int32[0, 8882, 7593, 6956, 6568, 6721, 6328, 5999, 5963, 5070  …  6, 6, 5, 5, 5, 5, 5, 5, 5, 5], Dict{String, UInt32}("focos" => 0x00002284, "#videos" => 0x00001535, "libertad" => 0x00000213, "ayudas" => 0x00000f7a, "henry" => 0x000018e2, "islamico" => 0x000018d5, "directivos" => 0x00001c4d, "soporte" => 0x000013cb, "astronautas" => 0x00002532, "advirtieron" => 0x000024f4…), 30244))

In [6]:
cloud(text, E, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, E, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

cloud_corpus (generic function with 1 method)

In [7]:
struct AWMD <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dcos::NormalizedCosineDistance
end

AWMD(X) = AWMD(X, [KnnResult(1) for _ in 1:Threads.nthreads()], NormalizedCosineDistance())

const PI_2 = convert(Float32, pi / 2)


@inline function nn_(X, res, dcos, u, V)
    res = reuse!(res)

    for (vid, _) in V
        push!(res, vid, evaluate(dcos, u, X[vid]))
    end
    
    minimum(res)
end

function onesidewmd(X, res, dcos, U, V)
    d::Float32 = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(X, res, dcos, X[uid], V))
        end
    end
    
    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::T, V::T) where T
    length(U) == 1 && haskey(U, 0) && return PI_2
    length(V) == 1 && haskey(V, 0) && return PI_2
    res = wmd.results[Threads.threadid()]
    dcos = wmd.dcos

    max(onesidewmd(wmd.X, res, dcos, U, V), onesidewmd(wmd.X, res, dcos, V, U))
end


In [8]:
db = VectorDatabase(cloud_corpus(D.corpus, E, T))
for c in eachcol(E.embeddings)
    normalize!(c)
end

dist = AWMD(MatrixDatabase(E.embeddings))

AWMD(MatrixDatabase{Matrix{Float32}}(Float32[0.14334421 0.13607663 … -0.07003834 0.09485001; 0.15824084 -0.06985427 … -0.15131347 -0.03590631; … ; 0.109540336 -0.094465375 … 0.09562941 0.15187858; 0.0069469837 -0.20757537 … -0.040153097 0.0046252417]), KnnResult[KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)  …  KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)], NormalizedCosineDistance())

In [9]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [10]:
index = ExhaustiveSearch(; db, dist);

In [11]:
let k = 11
    Qvectors = VectorDatabase(cloud_corpus(Q.corpus, E, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    s1 = scores(Q.labels, knn(I, D.labels))
    
    @htl """
    <div>search time: $t1, scores: $s1</div>
    """
end

<div>search time: 201.418880058, scores: <span class="Core-NamedTuple">(macrof1 = 0.12561720216080943, macrorecall = 0.137176973580315, accuracy = 0.17137756620053554)</span></div>

In [12]:
index = ParallelExhaustiveSearch(; db, dist);

In [13]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [14]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = cloud(qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [15]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "calentamiento global", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.140255 seconds (120.94 k allocations: 2.768 MiB)


1,19737,0.045,azucenau,"La jefa de gobierno de la Ciudad de México, Claudia Sheinbaum, aseguró que ella y el presidente de la República, Andrés Manuel López Obrador, “somos parte del mismo movimiento”, y que jamás será líder de oposición, pues “somos parte de la Cuarta Transformación”. https://t.co/FYoh0GuROr"
2,27745,0.066,Reforma,"El Presidente Andrés Manuel López Obrador propuso al Gobierno de #NuevoLeon acordar con empresarios que produzcan menos, usen menos #agua, y que así vaya más líquido a la gente. https://t.co/9kW1cPZBmj"
3,9748,0.076,SinEmbargoMX,"🚨 #URGENTE | El Gobierno de EU ofrece 5 millones de dólares por información que conduzca al arresto y condena de los mexicanos Ovidio Guzmán López, Iván Archivaldo Guzmán Salazar, Jesús Alfredo Guzmán Salazar y Joaquín Guzmán López, hijos de “El Chapo” https://t.co/S8eyJEDNTX https://t.co/YSXPkhUM7k"
4,19047,0.084,SinEmbargoMX,"El Secretario de Gobernación, Adán Augusto López, informó que el Gobierno federal y las aerolíneas llegaron a acuerdos para que los vuelos de carga doméstica y las operaciones tipo chárter migren “casi de inmediato” al AIFA https://t.co/LOJTQIPLxG https://t.co/41PU4vxjFX"
5,18128,0.084,abrahamendieta,"RT @abrahamendieta: Andrés Manuel sabe que los logros de este gobierno son conocidos por la gente, como demuestran las encuestas, y por eso…"
6,1610,0.089,PublimetroCol,"Prueba PCR negativa a viajeros y vacunación sin barreras, las sugerencias de Claudia López al Gobierno https://t.co/SceaZzIrUq"
7,3093,0.091,RicardoAlemanMx,"Si tenían duda, el @TEPJF_informa también está sometido por el gobierno de @lopezobrador_ ! Por eso hoy voto contra la decisión del @INEMexico ! Tienen razón Vargas Llosa y Ernesto Zedillo; con López, México vive la destrucción de su democracia! Al tiempo!"


  0.067016 seconds (120.96 k allocations: 2.769 MiB)


1,19869,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/jbAcSdIuPI"
2,10449,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/BNAdU3bp0n"
3,14765,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/cR2d88sSeF"
4,10549,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa88meses https://t.co/Yg7Rf8yDoS"
5,26176,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/cxi6tlCTMA"
6,25278,0.03,epigmenioibarra,"29 JoséE 30 JoséLuisL 31 Jhosivani 32 JulioCésarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa90Meses https://t.co/43JRhtAr00"
7,18240,0.03,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa85meses https://t.co/KH0xDLkhWz"


  0.090047 seconds (120.94 k allocations: 2.768 MiB)


1,639,0.068,bbcmundo,Cambio climático: 5 buenas noticias sobre la lucha contra el calentamiento global (contadas por científicos y expertos) https://t.co/VOKUupHVGN
2,2189,0.085,CNNEE,"▶ Todavía podemos mitigar el daño al planeta por el calentamiento global si exigimos políticas públicas a gobiernos, dicen expertas. https://t.co/7eoc24sboY"
3,6080,0.104,SinEmbargoMX,"#Opinión | Ana Ortega de @THL_Latino ""El consumo de carne o lo que no se habla del calentamiento global"" Si no adoptamos una postura más firme ante el calentamiento global, el panorama de México está a punto de ponerse sombrío https://t.co/HL8v7EBp2x https://t.co/k8Az8hDr5y"
4,27948,0.131,bbcmundo,El calentamiento global avanza y los países parecen incapaces de cumplir con los objetivos de emisiones. https://t.co/xW9mKglT2u
5,24021,0.135,bbcmundo,China es un actor clave en la COP26 y en la lucha contra el calentamiento global. https://t.co/JBwj6YmKxA
6,8099,0.169,bbcmundo,¿Cómo impacta en el calentamiento global una simple búsqueda en Internet? https://t.co/08Nau1gbBn
7,13719,0.193,Milenio,▶ ¿Era fuera de lugar? El polémico gol de Aldo Rocha en la final Atlas vs León La anotación se dio por válida y los Zorros empataron el marcador global https://t.co/5QCSfdz0YW https://t.co/omdW15uzOI


  0.067325 seconds (120.93 k allocations: 2.768 MiB)


1,16604,0.126,EFEnoticias,"Carlo Ancelotti, técnico italiano del Real Madrid, sigue dando positivo en coronavirus y no integra la expedición a Londres. #UCL https://t.co/m8dp8nJVVu"
2,22527,0.138,abc_es,"🔴 #ÚLTIMAHORA Don Juan Carlos da negativo en la PCR, tras el positivo en Covid de Rafa Nadal https://t.co/P9Li2qJubQ"
3,15179,0.143,EFEnoticias,@EFEdeportes Australia argumenta que Djokovic ha ignorado las medidas contra la covid. https://t.co/b9ffIVOqmZ
4,10320,0.143,CNNEE,▶ Djokovic aseguró en una entrevista que estaría dispuesto a no participar en más torneos por su postura sobre la vacuna contra el covid-19. Escucha sus palabras exactas: https://t.co/IirU4bbXO0
5,18921,0.147,EFEnoticias,"España se ha situado como el séptimo país por el volumen de la producción científica relacionada con la covid-19, con cifras similares a las de una ""potencia"" en esta materia como Alemania y por encima de otros países como Australia o Francia. https://t.co/38ZxpQ3Zxv"
6,8399,0.147,PublimetroCol,"¡Fuerza, Cami! Osorio, positiva por covid y es duda para el Abierto de Australia 😢 https://t.co/1icCPGpBaO"
7,24587,0.151,AristeguiOnline,Castillo con coronavirus como corona rompe récord mundial Guinness | #Video https://t.co/SaJIKYierX https://t.co/FmV7YOWKEX


In [16]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.090693 seconds (120.99 k allocations: 2.770 MiB)


1,4186,0.0,LaRazon_mx,#QuéTeParece | Cardenal dice que los medios también tienen culpa de pederastia en la Iglesia https://t.co/QEIjrW0P7s https://t.co/f4bqnN2bMC
2,25043,0.042,teleSURtv,#ENVIVO | El presidente de #Venezuela🇻🇪 @NicolasMaduro: y los venezolanos estén donde estén sepan muy bien que tienen aquí en #Venezuela su patria que abre los brazos para recibirlos cuando quieran volver a vivir en paz Síguelo aquí➡️https://t.co/tqMKHfl0b4 https://t.co/Ey7lVUtmaA
3,28668,0.043,SinEmbargoMX,"🚨 #URGENTE | José Ramón Cossío dijo que no sabe de dónde AMLO le agarró tirria, y que a lo mejor hay alguien que lo mal aconseja, pues conoce a los X. González, pero no se relaciona con ellos https://t.co/DNvQu1wpg3 https://t.co/IztRPMlghZ"
4,17519,0.044,julioastillero,"🚢 #EnVivo en #AstilleroInforma La reunión fue positiva, siento que al final hubo un reconocimiento que por parte de Semarnat se cometió un error en los canales de comunicación. Guardianes de la Sierra siempre estamos dispuestos al diálogo: Edwin Michel https://t.co/zgcQPfkHjp https://t.co/m4RPJZlD1P"
5,24936,0.044,CNNEE,"ANÁLISIS | Las diferentes crisis en EE.UU. mezcladas con golpes económicos y pronósticos de que lo peor está por venir, difícilmente es la normalidad que el presidente Joe Biden prometió el año pasado cuando dijo que “Estados Unidos está de regreso” https://t.co/8hbh2MuL3Q"
6,9496,0.044,SinEmbargoMX,"#VIDEO | No se tiene un reporte si hubo personas lesionadas o detenidas. Por medio de redes sociales, los usuarios pidieron a las autoridades capitalinas realizar una investigación de los hechos realizados en el paradero https://t.co/Iq59IiAm8G https://t.co/DnaC3oOzcX"
7,15718,0.044,teleSURtv,"#HILO 🧵| Para solicitar la #eutanasia, el paciente debe cubrir ciertos requerimientos, además que debe pasar por una asamblea de expertos que determinarán si es viable la #muertedigna, tras este análisis la persona puede detener el proceso cuando así lo desee https://t.co/6uWxk0BEO0"


  0.121699 seconds (120.94 k allocations: 2.768 MiB)


1,18375,0.0,bbcmundo,Un territorio multinacional y multiétnico. https://t.co/wf9GNtxFqx
2,12010,0.043,teleSURtv,"#Pakistán🇵🇰 pidió al #Gobierno de #Afganistán🇦🇫 en #Kabul que tome ""medidas severas"" contra los militantes que lanzan #ataques desde el interior del territorio afgano, un día después de que el ejército paquistaní matara a seis afganos https://t.co/Pvqsqr442F"
3,18664,0.043,TelemundoNews,🌎Descubren la isla más al norte de todo el planeta. Un equipo de científicos llegó por error a este territorio hasta ahora desconocido. 👇🏽 https://t.co/muAmeSbb1C
4,25866,0.049,Milenio,"▶️ Zacatecas es un punto estratégico para el crimen organizado, razón por la que grupos antagónicos se disputan el territorio El Ejército ha recuperado algunas comunidades que ahora están abandonadas por sus habitantes 📺 Los detalles con @azucenau #AzucenaALas10 https://t.co/IcQMfs6N3y"
5,21624,0.058,abc_es,Ayuso celebra que en los próximos cuatro años Castilla y León será un territorio «socialista free» https://t.co/kVAXEJYU5P
6,18181,0.059,el_pais,"RT @ElViajero_Pais: Un territorio vacío pero lleno de tesoros del arte mudéjar, castillos legendarios y maravillas geológicas, además de fó…"
7,24655,0.064,bbcmundo,"En esta versión moderna de David contra Goliat, el pequeño territorio guarda un as en la manga. https://t.co/E1zMz2bbUk"


  0.173248 seconds (120.96 k allocations: 2.769 MiB)


1,23465,0.0,PublimetroMX,El choque ocurrió a la altura de la estación El Vergel del Tren Ligero sobre Calzada de Tlalpan https://t.co/TqOlb51tdE
2,4572,0.04,azucenau,Sobre el accidente esta mañana en las obras del Aeropuerto de Santa Lucía la Sedena informa que sucedió cuando los trabajadores eran llevados en un vehículo tipo grúa y al saltar un tope 22 de ellos cayeron de una altura de 1.20 metros. Resultaron policontundidos. Ninguno grave. https://t.co/jhooGpmSIR
3,25022,0.041,El_Universal_Mx,"""La red de trata que la muerte de Evelin puso en evidencia, y que opera sobre todo al oriente de Morelos, puede formar parte del grupo criminal conocido como el Comando Tlahuica, que encabeza Humberto Figueroa Meza"" #Opinión de @hdemauleon https://t.co/Nv6bEc30Jg"
4,19027,0.042,TelemundoNews,📌 Lo último sobre el colapso del edificio en Surfside: reportan 99 personas desaparecidas. 👉🏻 Sigue la rueda de prensa de autoridades locales con los últimos detalles sobre las víctimas. https://t.co/NOELBADFxD
5,7004,0.042,azucenau,"El Gobierno capitalino acusó contradicciones y falta de rigor en el tercer informe causa raíz de la empresa noruega DNV sobre el desplome de un tramo de la línea 12 del Metro, por lo que fue rechazado por la Secretaría de Gestión Integral de Riesgos y Protección Civil. https://t.co/jlrqagFbDU"
6,16844,0.043,azucenau,"#AzucenaALas10 | La secretaría de seguridad ciudadana detuvo en la alcaldía Benito Juárez a Adán Alejandro ""N"" relacionado con el ataque e intento de asalto al hijo del embajador de México en Bélgica cuando se encontraba a bordo de su auto sobre avenida Revolución https://t.co/CkRQ5oZUzM"
7,12074,0.043,Milenio,▶ ¿Terrorismo o ataque directo? El gobernador de Guanajuato cambió de parecer sobre lo ocurrido en Salamanca y desechó la teoría de un acto terrorista ¿Qué se sabe hasta ahora del incidente y las víctimas? 📺 La historia con @azucenau #AzucenaxMILENIO https://t.co/mA7guKgHVV


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^\textsf{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Defina $H^\textsf{TF}_d$, donde $TF$ es la frecuencia de término normalizada.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
